In [1]:
from collections import namedtuple
from pathlib import Path

import torch
import trimesh
import numpy as np

In [2]:
class_labels = dict(
    head=0,
    torso=1,
    left_arm=2,
    left_hand=3,
    right_arm=4,
    right_hand=5,
    left_upper_leg=6,
    left_lower_leg=7,
    left_foot=8,
    right_upper_leg=9,
    right_lower_leg=10,
    right_foot=11,
)

class_colors = dict(
    head=np.array([255, 0, 0, 255]),
    torso=np.array([255, 0, 255, 255]),
    left_arm=np.array([255, 255, 0, 255]),
    left_hand=np.array([255, 128, 0, 255]), 
    right_arm=np.array([0, 255, 0, 255]),
    right_hand=np.array([0, 255, 128, 255]),
    left_upper_leg=np.array([0, 255, 255, 255]),
    left_lower_leg=np.array([0, 128, 255, 255]),
    left_foot=np.array([0, 0, 255, 255]),
    right_upper_leg=np.array([128, 0, 255, 255]),
    right_lower_leg=np.array([128, 255, 0, 255]),
    right_foot=np.array([255, 0, 128, 255])
)

In [3]:
BodyPart = namedtuple("body_part", ["label", "mesh"])

In [4]:
complete_mesh_fname = Path("../datasets/faust/MPI-FAUST/training/registrations/tr_reg_000.ply")
complete_mesh = trimesh.load_mesh(complete_mesh_fname, process=False)

In [5]:
body_parts_dir = Path("../datasets/faust/semantic_labels/")
body_parts = [
    BodyPart(mesh_fname.stem, trimesh.load_mesh(mesh_fname, process=False))
    for mesh_fname in body_parts_dir.glob("*.ply")
]

In [6]:
def find_vertices_on_mesh(vertices, mesh):
    def _find_vertex_on_mesh(_vertex):
        try:
            return np.where(np.all(mesh.vertices == _vertex, axis=1))[0][0]
        except IndexError:
            return None
    
    mesh_vertex_idx = []
    for vertex in vertices:
        idx = _find_vertex_on_mesh(vertex)
        if idx is not None:
            mesh_vertex_idx.append(idx)

    return mesh_vertex_idx

In [7]:
def _subtract_faces(vertex_idx_to_subtract, faces):
    def _to_subtract(face):
        return any(vertex_idx in vertex_idx_to_subtract for vertex_idx in face)

    faces_to_keep = [
        not _to_subtract(face)
        for face in faces
    ]
    return faces[faces_to_keep, :]


def remap_faces(faces_to_keep, remapping):
    return np.vectorize(remapping.__getitem__)(faces_to_keep)

def subtract_submesh(mesh, sub_mesh):
    idx_to_subtract = set(find_vertices_on_mesh(sub_mesh.vertices, mesh))
    idx_complete_mesh = set(range(len(mesh.vertices)))
    non_submesh_vertex_idx = idx_complete_mesh.difference(idx_to_subtract)

    remapping = dict(zip(non_submesh_vertex_idx, range(len(non_submesh_vertex_idx))))
    faces_to_keep = _subtract_faces(idx_to_subtract, np.asarray(mesh.faces))
    subtracted_mesh_faces = remap_faces(faces_to_keep, remapping)
    return trimesh.base.Trimesh(
        vertices=mesh.vertices[list(non_submesh_vertex_idx), :],
        faces=subtracted_mesh_faces,
        process=False,
    )

In [8]:
# subtracted_mesh = complete_mesh
# for _, sub_mesh in body_part_meshes:
#     subtracted_mesh = subtract_submesh(subtracted_mesh, sub_mesh)

In [9]:
test_mesh_fname = Path("../datasets/faust/MPI-FAUST/training/registrations/tr_reg_068.ply")
test_mesh = trimesh.load_mesh(test_mesh_fname, process=False)

In [10]:
for body_part in body_parts:
    body_part_idx = find_vertices_on_mesh(
        body_part.mesh.vertices,
        complete_mesh,
    )
    body_part_color = class_colors[body_part.label]
    test_mesh.visual.vertex_colors[body_part_idx] = body_part_color

In [11]:
test_mesh.show()

In [12]:
def generate_segmentation_labels(full_body_mesh, body_parts, class_labels):
    total_num_vertices = full_body_mesh.vertices.shape[0]
    semantic_labels = torch.full((total_num_vertices, 1), fill_value=-1, dtype=torch.int32)
    
    for body_part in body_parts:
        full_mesh_vertex_idx = find_vertices_on_mesh(body_part.mesh.vertices, full_body_mesh)
        semantic_class_idx = class_labels[body_part.label]
        semantic_labels[full_mesh_vertex_idx] = torch.tensor([semantic_class_idx], dtype=torch.int32)
    return semantic_labels

In [14]:
segmentation_labels = generate_segmentation_labels(complete_mesh, body_parts, class_labels)

In [17]:
np.savez("../datasets/faust/semantic_labels/segementations.npz", segmentation_labels=segmentation_labels.numpy())

In [19]:
test = np.load("../datasets/faust/semantic_labels/segementations.npz")

In [22]:
np.all(test["segmentation_labels"] == segmentation_labels.numpy())

True